This is a demo of how to use 'rok_estimator.py'.

In [1]:
from estimator.rok_estimator import *

In [2]:
# Specify parameters related to the underlying ring
ring_60 = RingParam(f=60)

# Create a new starting relation
rel = Relation(ring=ring_60,wdim=60,log_beta_wit_inf=0)
rel.show()

# Running pi_bdecomp reducing rel to rel_bdecomp
rel_bdecomp = rel.pi_bdecomp(ell=2)
rel_bdecomp.show(brief=True)

# Running pi_split reducing rel into rel_split
rel_split = rel.pi_split(2)
rel_split.show(brief=True)

# Running pi_batch reducing rel into rel_batch
rel_batch = rel.pi_batch()
rel_batch.show(brief=True)

# Running pi_fold reducing rel into rel_fold
rel_fold = rel.pi_fold(23)
rel_fold.show(brief=True)

# Running pi_norm reducing rel into rel_norm
rel_norm = rel.pi_norm()
rel_norm.show(brief=True)

Relation:
    H * F * W = Y
Statement:
    H: n_compress x (n_commit + n_rel)
    F: (n_commit + n_rel) x wdim
    Y: n_compress x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2 measured in max column canonical ell_2-norm
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    n_compress = 40, n_commit = 40, n_rel = 0
    wdim = 60, rep = 1
    log_2-norm (real | extr) = (8 |0 ), log_inf-norm (real | extr) = (0|0)
    wit size = 120.0 B 
 
Relation: wdim =       60, rep =   2, log_2-norm (real | extr) = (  8 |  0 ), log_inf-norm (real | extr) = (  0 |  0 ), wit size = 240.0 B , communication (growth | total) = (5.000 KB | 5.000 KB), soundness error (growth | total) = (2^-Infinity | 2^-Infinity)
Relation: wdim =       30, rep =   2, log_2-norm (real | extr) = (  8 |  0 ), log_inf-norm (real | extr) = (  0 |  0 ), wit size = 120.0 B , communication (growth | total) = (5.000 KB | 5.000 KB), soundness error (growth | total) = (2^-128 | 2^-128)


NameError: name 'deepcopy' is not defined

In [ ]:
# Simulate the execution of a sequence of RoKs.
# In the following example, the underlying ring is chosen to be the 60-th cyclotomic ring. 
# The SIS norm bound is 40-bit and the modulus is 64-bit. 
# The initial relation is a 2**22-by-25 matrix of binary ring elements which can be represented in 200 MB.
# The canonical ell_2-norms of the initial and extracted witnesses are both 2**16. 
# The estimated proof size is 17.21 MB.

ring_params = {
    "f": 60,
    "log_beta_sis_2": 40,
    "log_q": 64,
}

rel_params = {
    "wdim": 2**22,
    "rep": 25,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "ell": 4,
    "d": 2,
}

opener = [("norm", {}), ("batch", {}), ("split", {"d": ops_params["d"]}), ("fold", {})]
loop = [("bdecomp", {"ell": ops_params["ell"]}), ("norm", {}), ("batch", {}), ("split", {"d": ops_params["d"]}), ("fold", {})]
ops = opener + 10 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show(brief=False)

Ring parameters:
    conductor f: 60, degree phi: 16, modulus q: 2^64, beta_sis_2: 2^40
    SIS module rank n_sis: 61, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 128
    |R_q| = 128.0 B, |R_q^(n_sis)| = 7.625 KB
 
Subtractive set parameters:
    cardinality: 12
    forward ell_2 expansion factor gamma_2: 2^0
    inverse ell_2 expansion factor theta_2: 2^4
    forward ell_inf expansion factor gamma_inf: 2^2
    inverse ell_inf expansion factor theta_inf: 2^5
 
Relation:
    H * F * W = Y
Statement:
    H: n_compress x (n_commit + n_rel)
    F: (n_commit + n_rel) x wdim
    Y: n_compress x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2 measured in max column canonical ell_2-norm
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    n_compress = 61, n_commit = 61, n_rel = 0
    wdim = 4194304, rep = 25
    log_2-norm (real | extr) = (16 |16 ), log_inf-norm (